# Relational Games: Row Match [`dev`]

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os; os.environ['TF_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

import data_utils
import sys; sys.path.append('../../')
import utils

2023-09-06 17:26:46.006130: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-06 17:26:46.832222: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-06 17:26:49.642043: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-06 17:26:49.642743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 17:27:22.214560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
tf.config.list_physical_devices('GPU')

[]

## Set Up

In [4]:
import argparse
args = argparse.Namespace()
args.task = '1task_match_patt'
args.train_split = 'pentos'
args.test_split_size = 5000
args.n_epochs = 100
args.early_stopping = True
args.batch_size = 256
args.val_size = 1000
args.test_size = 5000

### Load Data

In [5]:
# utils.print_section('LOADING DATA')
data_path = '../../data/relational_games'

dataset_specs = np.load(f'{data_path}/dataset_specs.npy', allow_pickle=True).item()

task_datasets = data_utils.load_task_datasets(args.task, data_path, data_format='tfrecord', ds_specs=dataset_specs)

train_split_ds = task_datasets[args.train_split]
# train_split_ds = data_utils.load_ds_from_npz(filename)

train_ds, val_ds, test_ds = utils.split_ds(train_split_ds, val_size=args.val_size, test_size=args.test_size)
del train_split_ds

batch_size = args.batch_size
val_ds = val_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

eval_datasets = {split: ds.take(args.test_split_size).batch(batch_size) for split, ds in task_datasets.items() if split != args.train_split}


In [6]:
len(test_ds) * batch_size

5120

In [7]:
len(train_ds)

244000

### Training Set up

In [8]:
#region training setup
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
create_opt = lambda: tf.keras.optimizers.Adam()
metrics = [
        tf.keras.metrics.BinaryAccuracy(name='acc'),
        # tf.keras.metrics.Precision(class_id=1, name='precision'),
        # tf.keras.metrics.Recall(class_id=1, name='recall'),
        # tf.keras.metrics.F1Score(average='weighted', name='f1'),
        tf.keras.metrics.AUC(curve='ROC', multi_label=True, name='auc')
        ]

from tqdm.keras import TqdmCallback
from tqdm import tqdm
def create_callbacks(data_size=None, batch_size=None):
    callbacks = [
        TqdmCallback(data_size=data_size, batch_size=batch_size, tqdm_class=tqdm),
        # wandb.keras.WandbMetricsLogger(log_freq='epoch')
    ]

    if args.early_stopping:
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, start_from_epoch=20))

    return callbacks

fit_kwargs = dict(epochs=args.n_epochs)
batch_size = args.batch_size
#endregion

/gpfs/gibbs/project/lafferty/ma2393/conda_envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CoRelNet

In [8]:
from relational_neural_networks.multi_head_relation import MultiHeadRelation
from relational_neural_networks.relational_graphlet_convolution import RelationalGraphletConvolution
from relational_neural_networks.grouping_layers import TemporalGrouping, FeatureGrouping
from cnn_embedder import CNNEmbedder

In [9]:
# global parameters
cnn_embedder_kwargs = dict(n_f=(16,16), s_f=(3,3), pool_size=2)

# RelConvNet
corelnet_mhr_kwargs = dict(rel_dim=1, proj_dim=4, symmetric=True)
cnn_embedder_kwargs = dict(n_f=(16,16), s_f=(3,3), pool_size=2)

# def create_relconvnet():
mhr = MultiHeadRelation(**corelnet_mhr_kwargs, name='mhr')
cnn_embedder = CNNEmbedder(**cnn_embedder_kwargs)

model = tf.keras.Sequential(
    [
        cnn_embedder,
        tf.keras.layers.LayerNormalization(axis=1), # TCN? axis=1 is the temporal axis
        mhr,
        tf.keras.layers.Softmax(axis=-1, name='softmax'),
        tf.keras.layers.Flatten(name='flatten'),
        tf.keras.layers.Dense(32, activation='relu', name='hidden_dense1'),
        tf.keras.layers.Dense(2, name='output')],
    name='corelnet')

    # return model

In [10]:
# opt = tf.keras.optimizers.Adam(tf.keras.optimizers.schedules.ExponentialDecay(0.01, 100, 0.96))
opt = tf.keras.optimizers.Adam(0.001)
model.compile(loss=loss, optimizer=opt, metrics=metrics) # compile
model.build(input_shape=(None, *train_ds.element_spec[0].shape)) # build
print(model.summary())

Model: "corelnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_embedder (CNNEmbedder)  (None, 9, 16)             2768      
                                                                 
 layer_normalization (Layer  (None, 9, 16)             18        
 Normalization)                                                  
                                                                 
 mhr (MultiHeadRelation)     (None, 9, 9, 1)           68        
                                                                 
 softmax (Softmax)           (None, 9, 9, 1)           0         
                                                                 
 flatten (Flatten)           (None, 81)                0         
                                                                 
 hidden_dense1 (Dense)       (None, 32)                2624      
                                                          

In [ ]:
# TODO: check if LayerNormalization w axis=1 is TCN
# TOOD: check the 'encoder' of CoRelNet

In [11]:
train_size = -1 # 5_000
batch_size = 256

shuffle_buffer_size = train_size
train_ds_sample = train_ds.take(train_size).batch(batch_size)
history = model.fit(
    train_ds_sample, validation_data=val_ds, verbose=1, epochs=50)

Epoch 1/50
954/954 [==============================] - 46s 37ms/step - loss: 0.6967 - acc: 0.5000 - auc: 0.4991 - val_loss: 0.6989 - val_acc: 0.5000 - val_auc: 0.5000
Epoch 2/50
953/954 [============================>.] - ETA: 0s - loss: 0.6940 - acc: 0.5000 - auc: 0.5000

: 

: 

In [ ]:
model.evaluate(test_ds)

20/20 [==============================] - 8s 81ms/step - loss: 0.0092 - acc: 0.6556 - precision: 0.6151 - recall: 0.6810 - f1: 0.9982 - auc: 0.6586


[0.009209304116666317,
 0.6556000113487244,
 0.6150537729263306,
 0.6809523701667786,
 0.9982000589370728,
 0.6585724353790283]

In [ ]:
utils.plot_history(history, plot_attrs=('loss','acc'));

In [ ]:
model.evaluate(val_ds)

## Relational Convolution

In [ ]:
from relational_neural_networks.multi_head_relation import MultiHeadRelation
from relational_neural_networks.relational_graphlet_convolution import RelationalGraphletConvolution
from relational_neural_networks.grouping_layers import TemporalGrouping, FeatureGrouping
from cnn_embedder import CNNEmbedder

In [ ]:
import itertools
# groups = [group for group in itertools.permutations(range(9), r=3) if all(x in (0,1,2,6,7,8) for x in group)]
groups = [(0,1,2), (6,7,8)]

In [ ]:
# RelConvNet
relconv_mhr_kwargs = dict(rel_dim=8, proj_dim=4, symmetric=True)
relconv_kwargs = dict(n_filters=32, graphlet_size=3,
        symmetric_inner_prod=False)
cnn_embedder_kwargs = dict(n_f=(16,16), s_f=(3,3), pool_size=2)

# def create_relconvnet():
mhr1 = MultiHeadRelation(**relconv_mhr_kwargs, name='mhr1')
mhr2 = MultiHeadRelation(**relconv_mhr_kwargs, name='mhr2')

rel_conv1 = RelationalGraphletConvolution(
    **relconv_kwargs, groups=groups, name='rgc1')
rel_conv2 = RelationalGraphletConvolution(
    **relconv_kwargs, groups='combinations', name='rgc2')

# feature_grouping = FeatureGrouping(num_groups=32, use_pos=True,
#     mlp_shape=(32, 32), mlp_activations='relu', name='feature_grouping')
temporal_grouper1 = TemporalGrouping(num_groups=16, weight_initializer='glorot_uniform', name='temporal_grouper1')
temporal_grouper2 = TemporalGrouping(num_groups=4, weight_initializer='glorot_uniform', name='temporal_grouper2')

cnn_embedder = CNNEmbedder(**cnn_embedder_kwargs)
l2_normalizer = tf.keras.layers.UnitNormalization(name='l2_normalization')

inputs = tf.keras.layers.Input(shape=train_ds.element_spec[0].shape)
embedded_objects = cnn_embedder(inputs)
embedded_objects = l2_normalizer(embedded_objects)
rel_tensor = mhr1(embedded_objects)
# g = feature_grouping(embedded_objects)
g = temporal_grouper1(embedded_objects)
convolution = rel_conv1(rel_tensor, g)
# rel_tensor = mhr2(convolution)
# g = temporal_grouper2(convolution)
# convolution = rel_conv2(rel_tensor, g)
# convolution = rel_conv(rel_tensor)

x = tf.keras.layers.Flatten(name='flatten')(convolution)
x = tf.keras.layers.Dense(64, activation='relu', name='hidden_dense1')(x)
x = tf.keras.layers.Dense(2, name='output')(x)
outputs = x

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='relconvent')

In [ ]:
# opt = tf.keras.optimizers.Adam(tf.keras.optimizers.schedules.ExponentialDecay(0.01, 10, 0.9)) #TODO: here
opt = tf.keras.optimizers.Adam(0.001) #TODO: here
# opt = tf.keras.optimizers.SGD(0.001)
model.compile(loss=loss, optimizer=opt, metrics=metrics) # compile
model.build(input_shape=(None, *train_ds.element_spec[0].shape)) # build
print(model.summary())

Model: "relconvent"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 9, 12, 12, 3)]       0         []                            
                                                                                                  
 cnn_embedder_5 (CNNEmbedde  (None, 9, 16)                2768      ['input_2[0][0]']             
 r)                                                                                               
                                                                                                  
 l2_normalization (UnitNorm  (None, 9, 16)                0         ['cnn_embedder_5[0][0]']      
 alization)                                                                                       
                                                                                         

In [ ]:
train_size = -1 # 5_000
batch_size = 512

shuffle_buffer_size = train_size
train_ds_sample = train_ds.take(train_size).batch(batch_size)
history = model.fit(
    train_ds_sample, validation_data=val_ds, verbose=1, epochs=50)

Epoch 1/50
477/477 [==============================] - 160s 312ms/step - loss: 0.6925 - acc: 0.5000 - auc: 0.5012 - val_loss: 0.6832 - val_acc: 0.5000 - val_auc: 0.5389
Epoch 2/50
477/477 [==============================] - 149s 300ms/step - loss: 0.4385 - acc: 0.5786 - auc: 0.6324 - val_loss: 0.2847 - val_acc: 0.6450 - val_auc: 0.6906
Epoch 3/50
477/477 [==============================] - 150s 304ms/step - loss: 0.2178 - acc: 0.6579 - auc: 0.6876 - val_loss: 0.1445 - val_acc: 0.6640 - val_auc: 0.6795
Epoch 4/50
477/477 [==============================] - 149s 302ms/step - loss: 0.1158 - acc: 0.6672 - auc: 0.6806 - val_loss: 0.0758 - val_acc: 0.6720 - val_auc: 0.6845
Epoch 5/50
187/477 [==========>...................] - ETA: 1:21 - loss: 0.0763 - acc: 0.6788 - auc: 0.6885

In [ ]:
model.evaluate(test_ds)

20/20 [==============================] - 8s 81ms/step - loss: 0.0092 - acc: 0.6556 - precision: 0.6151 - recall: 0.6810 - f1: 0.9982 - auc: 0.6586


[0.009209304116666317,
 0.6556000113487244,
 0.6150537729263306,
 0.6809523701667786,
 0.9982000589370728,
 0.6585724353790283]

In [ ]:
utils.plot_history(history, plot_attrs=('loss','acc'));

## 2-Layer RelConvNet

In [9]:
from relational_neural_networks.multi_head_relation import MultiHeadRelation
from relational_neural_networks.relational_graphlet_convolution import RelationalGraphletConvolution
from relational_neural_networks.grouping_layers import TemporalGrouping, FeatureGrouping
from cnn_embedder import CNNEmbedder

In [16]:
relconv_mhr_kwargs = dict(rel_dim=16, proj_dim=4, symmetric=True)
relconv_mhr2_kwargs = dict(rel_dim=16, proj_dim=4, symmetric=True)
relconv_kwargs = dict(n_filters=16, graphlet_size=3,
        symmetric_inner_prod=False)
relconv2_kwargs = dict(n_filters=16, graphlet_size=5, symmetric_inner_prod=False)
cnn_embedder_kwargs = dict(n_f=(16,16), s_f=(3,3), pool_size=2)

In [17]:
# import itertools
# groups = [group for group in itertools.combinations(range(9), r=3) if all(x in (0,1,2,6,7,8) for x in group)]
# # groups = [(0,1,2), (6,7,8)]
# groups

In [20]:
import itertools
s1 = 3
groups1 = [group for group in itertools.combinations(range(6), r=s1)]
n1 = len(groups1)

s2 = 5
n2 = 20
groups2 = [group for group in itertools.combinations(range(n1), r=s2)]
groups2_choice_idx = np.random.choice(len(groups2), n2, replace=False)
groups2 = [group for i,group in enumerate(groups2) if i in groups2_choice_idx]

In [21]:
# LAYERS
mhr1 = MultiHeadRelation(**relconv_mhr_kwargs, name='mhr1')
mhr2 = MultiHeadRelation(**relconv_mhr2_kwargs, name='mhr2')

cnn_embedder = CNNEmbedder(**cnn_embedder_kwargs)
l2_normalizer = tf.keras.layers.UnitNormalization(name='l2_normalization')
get_objs_layer = tf.keras.layers.Lambda(lambda x: tf.gather(x, [0,1,2,6,7,8], axis=1), name='get_objects')

# def create_relconvnet():
mhr1 = MultiHeadRelation(**relconv_mhr_kwargs, name='mhr1')
mhr2 = MultiHeadRelation(**relconv_mhr2_kwargs, name='mhr2')

rel_conv1 = RelationalGraphletConvolution(
    **relconv_kwargs, groups=groups1, name='rgc1')
rel_conv2 = RelationalGraphletConvolution(
    **relconv2_kwargs, groups=groups2, name='rgc2')

# feature_grouping = FeatureGrouping(num_groups=32, use_pos=True,
#     mlp_shape=(32, 32), mlp_activations='relu', name='feature_grouping')
# temporal_grouper1 = TemporalGrouping(num_groups=16, weight_initializer='glorot_uniform', name='temporal_grouper1')
# temporal_grouper2 = TemporalGrouping(num_groups=4, weight_initializer='glorot_uniform', name='temporal_grouper2')

cnn_embedder = CNNEmbedder(**cnn_embedder_kwargs)
l2_normalizer = tf.keras.layers.UnitNormalization(name='l2_normalization')

# MODEL DEFINITION
inputs = tf.keras.layers.Input(shape=train_ds.element_spec[0].shape)
embedded_objects = cnn_embedder(inputs)
embedded_objects = l2_normalizer(embedded_objects)

rel_tensor = mhr1(embedded_objects)
# g = feature_grouping(embedded_objects)
# g = temporal_grouper1(embedded_objects)
convolution = rel_conv1(rel_tensor)
# convolution = rel_conv1(rel_tensor, g)

convolution = l2_normalizer(convolution)

rel_tensor = mhr2(convolution)
convolution = rel_conv2(rel_tensor)
# g = temporal_grouper2(convolution)
# convolution = rel_conv2(rel_tensor, g)

x = tf.keras.layers.Flatten(name='flatten')(convolution)
x = tf.keras.layers.Dense(64, activation='relu', name='hidden_dense1')(x)
x = tf.keras.layers.Dense(2, name='output')(x)
outputs = x

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='relconvent')

In [22]:
opt = tf.keras.optimizers.Adam(0.001) #TODO: here
model.compile(loss=loss, optimizer=opt, metrics=metrics); # compile
model.build(input_shape=(None, *train_ds.element_spec[0].shape)); # build
model.summary()

Model: "relconvent"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 9, 12, 12, 3)]       0         []                            
                                                                                                  
 cnn_embedder_5 (CNNEmbedde  (None, 9, 16)                2768      ['input_3[0][0]']             
 r)                                                                                               
                                                                                                  
 l2_normalization (UnitNorm  multiple                     0         ['cnn_embedder_5[0][0]',      
 alization)                                                          'rgc1[0][0]']                
                                                                                         

In [23]:
train_size = -1 # 5_000
batch_size = 512

shuffle_buffer_size = train_size
train_ds_sample = train_ds.take(train_size).batch(batch_size)
history = model.fit(
    train_ds_sample, validation_data=val_ds, verbose=1, epochs=50)

Epoch 1/50


2023-09-06 14:20:34.033149: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 201944 of 250000


  1/477 [..............................] - ETA: 4:12:49 - loss: 0.7148 - acc: 0.5000 - auc: 0.4973

2023-09-06 14:20:36.116577: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


477/477 [==============================] - ETA: 0s - loss: 0.6936 - acc: 0.5000 - auc: 0.5004

...

2023-09-06 15:46:33.707622: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


477/477 [==============================] - ETA: 0s - loss: 0.6926 - acc: 0.5000 - auc: 0.5112

2023-09-06 15:48:08.120720: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 142541 of 250000
2023-09-06 15:48:14.282359: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


477/477 [==============================] - 114s 212ms/step - loss: 0.6926 - acc: 0.5000 - auc: 0.5112 - val_loss: 0.6919 - val_acc: 0.5000 - val_auc: 0.5281
Epoch 46/50


2023-09-06 15:48:25.102918: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 189037 of 250000


*NOTES:*
- need intermediate layer before final output
- todo: check if grouping layer helps
- todo: experiment w given good discrete groups